# Download All The Images

You need a SpaceWarps installation plus pickles in order to be able to do the first step of reducing to a catalog. Sorry.

I skipped stage1 tests that were still undecided. I think those go into stage 2. This saves a lot of images probably.

## TODO:
- Incorporate skill into outlier_clusters_dbscan
- add check_make for the different directories
- add object detection for all objects in field as well as click locations
- paths for catalogs etc are now messed up

- put in the real lenses

- remake cutouts using just sims and duds from stage1 as well as knownlens, and everything from stage2
- next time when making the clusters make the filenames and alpha statuses too.

In [33]:
# Here you specify some paths for files!
project_path = '/Volumes/Seagate/cs231n/'
# pickles path
base_collection_path = project_path + 'swap_pickles/09.02.15/'
# catalog path
annotated_catalog_path = project_path + 'catalog/annotated_catalog.csv'
# repeat without annotations
unannotated_catalog_path = project_path + 'catalog/unannotated_catalog.csv'
# cluster catalog path
cluster_catalog_path = project_path + 'catalog/cluster_catalog.csv'
# images path
images_path = project_path + 'images/fields/'
# cutouts path
cutouts_path = project_path + 'images/cutouts/'
# cache dir for clustering
cachedir = '../.cache'
# knownlens catalog
knownlens_path = project_path + 'knownlens/knownlens.csv'

In [12]:
# Here are some other config parameters
categories = ['ID', 'ZooID', 'location', 'mean_probability', 'category', 'kind', 'flavor', 'state', 'status', 'truth',
              'stage']
# annotation_categories = ['At_X', 'At_Y', 'ItWas', 'Name', 'PD', 'PL']
annotation_categories = ['At_X', 'At_Y', 'PD', 'PL']
cluster_catalog_labels = ['cluster_label', 'cluster_ID', 'x', 'y', 'num_markers', 'skill_sum', 'dist_within',
                          'cutoutname', 'fieldname']

eps = int(96 * 0.5)
min_samples = 2
stamp_size = eps * 2

In [4]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import pairwise_distances
from sklearn.externals import joblib
import pandas as pd
import numpy as np
from numpy import log2, ndarray

# ======================================================================


def outlier_clusters_dbscan(x, y, skill=None, memory=None,
                            eps=25, min_samples=5,
                            convert_outliers=True):
    """
    DBSCAN parameters
    eps : float, optional
        The maximum distance between two samples for them to be considered
        as in the same neighborhood.
    min_samples : int, optional
        The number of samples in a neighborhood for a point to be considered
        as a core point.
        
    convert_outliers : binary, optional
        DBSCAN assigns outliers a cluster label -1. This says take each -1 and give it a unique positive value
    """
    
    # TODO: incorporate skill
    data = np.vstack((x, y)).T

    if len(data) == 0:
        # uh.
        print('clustering: NO cluster members!')
        cluster_centers = np.array([[-1, -1]])
        cluster_labels = []
        labels = []
        n_clusters = 0
        dist_within = np.array([])

    elif len(data) == 1:
        #print('clustering: only 1 data point!')
        cluster_centers = data
        cluster_labels = [0]
        labels = np.array([0])
        n_clusters = 1
        dist_within = np.array([0])

    else:

        clusterer = DBSCAN(eps=eps, min_samples=min_samples)
        db = clusterer.fit(data)
        labels = db.labels_
        n_clusters = len(set(labels))
        cluster_labels = list(set(labels))
        if convert_outliers:
            # now step through all the labels and set the -1s each to a new unique label
            label_max = np.max((max(cluster_labels) + 1, 101))
            for label_i, label in enumerate(labels):
                if label == -1:
                    labels[label_i] = label_max
                    label_max += 1
            cluster_labels = list(set(labels))
        # cludgey
        cluster_centers = np.array([np.mean(data[labels == i], axis=0)
                                    for i in cluster_labels])
        

    # print n_clusters
    # print labels

    # cludgey
    dist_within_final = np.array([np.max(pairwise_distances(
            data[labels == i])) for i in cluster_labels])

    return cluster_centers, cluster_labels, labels, n_clusters, dist_within_final

outlier_clusters = outlier_clusters_dbscan

def shannon(x):

    if isinstance(x, ndarray) == False:

        if x>0:
            res = x*log2(x)
        else:
            res = 0.0
    
    else:
        x[x == 0] = 1.0
        res = x*log2(x)

    return res

def expectedInformationGain(p0, M_ll, M_nn):

    p1 = 1-p0

    I =   p0 * (shannon(M_ll) + shannon(1-M_ll)) \
        + p1 * (shannon(M_nn) + shannon(1-M_nn)) \
        - shannon(M_ll*p0 + (1-M_nn)*p1) \
        - shannon((1-M_ll)*p0 + M_nn*p1)

    return I

In [5]:
from scipy.ndimage import imread
from scipy.misc import imsave
from os import path
from urllib import FancyURLopener

# the fancy way of scraping images from the web; you gotta pretend you are a browser
class MyOpener(FancyURLopener):
    version = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11'
myopener = MyOpener()

def get_online_png(url, outname):
    fname = url.split('/')[-1]

    # download file if we don't already have it
    if not path.exists(outname):
        F = myopener.retrieve(url, outname)
    else:
        # TODO: this is glitched?!
        F = [outname]
    I = imread(F[0]) * 1. / 255
    return I

## reduce pickles to catalog

A catalog of stage1 and stage2 field objects with:
    - ID (for later retrieval)
    - probability from users
I could probably reduce the following into one or two parameters
    - category
    - kind
    - flavor
    - state
    - status
    - truth
When I make the cutouts I will also need:
    - locations of clicks and associated skills

In [ ]:
import swap
stages = range(1, 3)


# TODO: incorporate knownlens info.
catalog = []
for stage in stages:
    print(stage)
    collection_path = base_collection_path + 'stage{0}'.format(stage) + '/CFHTLS_collection.pickle'
    collection = swap.read_pickle(collection_path, 'collection')
    for ID in collection.list():

        subject = collection.member[ID]
        catalog_i = []
        
        # for stage1 we shall skip the tests for now
        if (stage == 1) * (subject.category == 'test'):
            continue
        
        # flatten out x and y. also cut out empty entries
        annotationhistory = subject.annotationhistory
        x_unflat = annotationhistory['At_X']
        x = np.array([xi for xj in x_unflat for xi in xj])

        # cut out catalogs with no clicks
        if len(x) < 1:
            continue
        # oh yeah there's that absolutely nutso entry with 50k clicks
        if len(x) > 10000:
            continue
        
        for category in categories:
            if category == 'stage':
                catalog_i.append(stage)
            else:
                catalog_i.append(subject.__dict__[category])
        for category in annotation_categories:
            catalog_i.append(list(annotationhistory[category]))
            
        catalog.append(catalog_i)
catalog = pd.DataFrame(catalog, columns=categories + annotation_categories)

# save catalog
catalog.to_csv(annotated_catalog_path)

# now repeat without annotations. Saves a lot of space!
catalog[categories].to_csv(unannotated_catalog_path)

In [9]:
check_types = ['category', 'kind', 'flavor', 'state', 'status', 'truth']
for check_type in check_types:
    print(check_type, np.unique(catalog[check_type]))

('category', array(['test', 'training'], dtype=object))
('kind', array(['dud', 'sim', 'test'], dtype=object))
('flavor', array(['dud', u'lensed galaxy', u'lensed quasar', u'lensing cluster',
       'test'], dtype=object))
('state', array(['active', 'inactive'], dtype=object))
('status', array(['detected', 'rejected', 'undecided'], dtype=object))
('truth', array(['LENS', 'NOT', 'UNKNOWN'], dtype=object))


Note: For evaluating the objects in catalog, like the annotation_categories, you will want to use ast.literal_eval(object)

## Create Cluster Catalog

In [ ]:
from ast import literal_eval

catalog = pd.read_csv(annotated_catalog_path)
memory = joblib.Memory(cachedir=cachedir)
# memory.clear()

cluster_catalog = []
ith = 0
cluster_ID = 0
# step through the objects in catalog
for cati in range(len(catalog)):
    
    # get markers
    x_unflat = literal_eval(catalog['At_X'][cati])
    y_unflat = literal_eval(catalog['At_Y'][cati])
    # flatten out x and y. also cut out empty entries
    x = np.array([xi for xj in x_unflat for xi in xj])
    y = np.array([yi for yj in y_unflat for yi in yj])
    
    # repeat for PD and PL = proxies for weight
    PL_everyone = literal_eval(catalog['PL'][cati])
    PL_unflat = []
    PD_everyone = literal_eval(catalog['PD'][cati])
    PD_unflat = []
    for i, xj in enumerate(x_unflat):
        # len(xj) of empty = 0
        PL_unflat.append([PL_everyone[i]] * len(xj))
        PD_unflat.append([PD_everyone[i]] * len(xj))

    PL = np.array([PLi for PLj in PL_unflat for PLi in PLj])
    PD = np.array([PDi for PDj in PD_unflat for PDi in PDj])
    skill = expectedInformationGain(0.5, PL, PD)

    # cluster
    cluster_centers, cluster_center_labels, cluster_labels, n_clusters, dist_within = \
        outlier_clusters(x, y, skill, memory=memory, eps=eps, min_samples=min_samples)
    
    # add to catalog
    for cluster_center_label_i, cluster_center_label in enumerate(cluster_center_labels):
        
        cluster_center = cluster_centers[cluster_center_label_i]
        x, y = cluster_center

        if np.int(np.ceil(x - stamp_size / 2)) >= 440:
            print('Mislabled cluster? {0} {1} {2} {3}'.format(cati, cluster_center_label_i, x, y))
            continue
        elif np.int(np.ceil(y - stamp_size / 2)) >= 440:
            print('Mislabled cluster? {0} {1} {2} {3}'.format(cati, cluster_center_label_i, x, y))
            continue
        elif np.int(np.floor(x + stamp_size / 2)) < 0:
            print('Mislabled cluster? {0} {1} {2} {3}'.format(cati, cluster_center_label_i, x, y))
            continue        
        elif np.int(np.floor(y + stamp_size / 2)) < 0:
            print('Mislabled cluster? {0} {1} {2} {3}'.format(cati, cluster_center_label_i, x, y))
            continue
        
        if cluster_center_label == -1 or cluster_center_label > 100:
            # outlier cluster
            # so really every point is its own cluster...
            D = 0
        else:
            D = dist_within[cluster_center_label]
            
        members = (cluster_labels == cluster_center_label)

        N0 = np.sum(members)
        S = np.sum(skill[members])
            
        # ['cluster_label', 'cluster_ID', 'x', 'y', 'num_markers', 'skill_sum', 'dist_within']
        
        fieldname = '{0}.png'.format(catalog['ZooID'][cati])
        cutoutname = '{1}_{0}.png'.format(catalog['ZooID'][cati], cluster_ID)
        cluster_catalog_i = [catalog[category][cati] for category in categories] + \
                            [cluster_center_label, cluster_ID, cluster_center[0], cluster_center[1], N0, S, D,
                             cutoutname, fieldname]
        cluster_catalog.append(cluster_catalog_i)
        
        cluster_ID += 1
    
 
cluster_catalog = pd.DataFrame(cluster_catalog, columns=categories + cluster_catalog_labels)


# save catalog
cluster_catalog.to_csv(cluster_catalog_path)

## Create Cutouts From Catalog

add an 'in_alpha' parameter to cluster catalog

In [26]:
print (clusteri, len(cluster_catalog))

(16314, 35083)


In [28]:
print(outname_field)

/Volumes/Seagate/cs231n/images/fields/ASW000526z.png


In [29]:
#cluster_catalog = pd.read_csv(cluster_catalog_path)
#alphas = []
for clusteri in range(16314 , len(cluster_catalog)):

    x = cluster_catalog['x'][clusteri]
    y = cluster_catalog['y'][clusteri]
    
    outname_field = images_path + cluster_catalog['fieldname'][clusteri]
    im = get_online_png(cluster_catalog['location'][clusteri], outname_field)       
        
    min_x = np.int(np.floor(x - stamp_size / 2))
    max_x = np.int(np.ceil(x + stamp_size / 2))
    min_y = np.int(np.floor(y - stamp_size / 2))
    max_y = np.int(np.ceil(y + stamp_size / 2))
    pad_min_x = 0
    pad_max_x = 0
    pad_min_y = 0
    pad_max_y = 0
    pad_trip = False
    if min_x < 0:
        pad_min_x = -min_x
        min_x = 0
        pad_trip = True
    if max_x >= im.shape[1]:
        pad_max_x = max_x - im.shape[1]
        max_x = im.shape[1]
        pad_trip = True
    if min_y < 0:
        pad_min_y = -min_y
        min_y = 0
        pad_trip = True
    if max_y >= im.shape[0]:
        pad_max_y = max_y - im.shape[0]
        max_y = im.shape[0]
        pad_trip = True
    
    im_cut = np.pad(im[min_y: max_y, min_x: max_x],
                    ((pad_min_y, pad_max_y), (pad_min_x, pad_max_x), (0, 0)),
                    mode='constant')
    im_cut = im_cut[:stamp_size, :stamp_size, :3]
    
    # now check if the center is in a high alpha region (if alpha is included, as it is with training)
    alpha = 0
    if im.shape[2] == 4:
        im_mask = im[:,:,3].copy()
                
        im_mask -= im_mask.mean()
        im_mask /= im_mask.std()
        im_mask[im_mask < 5] = 0
        im_mask[im_mask >= 5] = 1
        
        if ((y >= 0) * (y < 440) *
            (x >= 0) * (x < 440)):
            if im_mask[y, x] == 1:
                alpha = 1
    alphas.append(alpha)
    
    outname_cluster = cutouts_path + cluster_catalog['cutoutname'][clusteri]
    
    imsave(outname_cluster, im_cut)
    
cluster_catalog['alpha'] = alphas
# save catalog
cluster_catalog.to_csv(cluster_catalog_path)

Since this took so long and I'm at 17 gb with something like 150k more clusters to go, let's stop and assess our distribution.

In [3]:
# now let's print some essential stats:
cc = cluster_catalog

print(cc.columns)

# get number of objects in stage 1 vs stage 2
print(cc.groupby(['stage']).apply(len))

print(cc.groupby(['kind']).apply(len))

print(cc.groupby(['stage', 'kind']).apply(len))

# get number of flavors
print(cc.groupby(['flavor']).apply(len))


# combine together
print(cc.groupby(['stage', 'flavor']).apply(len))
print(cc.groupby(['stage', 'flavor', 'alpha']).apply(len))
print(cc.groupby(['stage', 'flavor', 'status']).apply(len))

# also show median probability
print(cc.groupby(['stage', 'flavor'])['mean_probability'].apply(np.mean))
print(cc.groupby(['stage', 'flavor', 'status'])['mean_probability'].apply(np.mean))

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'ID', u'ZooID', u'location', u'mean_probability', u'category', u'kind', u'flavor', u'state', u'status', u'truth', u'stage', u'cluster_label', u'cluster_ID', u'x', u'y', u'num_markers', u'skill_sum', u'dist_within', u'cutoutname', u'fieldname', u'alpha'], dtype='object')
stage
1        24177
2        10906
dtype: int64
kind
dud      9491
sim     16562
test     9030
dtype: int64
stage  kind
1      dud      8362
       sim     15815
2      dud      1129
       sim       747
       test     9030
dtype: int64
flavor
dud                9491
lensed galaxy      5983
lensed quasar      5271
lensing cluster    5308
test               9030
dtype: int64
stage  flavor         
1      dud                8362
       lensed galaxy      5286
       lensed quasar      5271
       lensing cluster    5258
2      dud                1129
       lensed galaxy       697
       lensing cluster      50
       test               9030
dtype: int64
stage  flavor           al

In [7]:
import pandas as pd
cluster_catalog = pd.read_csv('/Users/cpd/Projects/strongcnn/catalog/cluster_catalog.csv')
print('Statistics for the cutouts:')
print(cluster_catalog.groupby(['stage', 'alpha', 'flavor']).apply(len))
print(cluster_catalog.groupby(['stage', 'alpha', 'kind']).apply(len))

# the fields are where the cutouts come from so let's get some stats from them, too
field_catalog = pd.read_csv('/Users/cpd/Projects/strongcnn/catalog/unannotated_catalog.csv')
print('Statistics for the fields:')
print(field_catalog.groupby(['stage', 'flavor']).apply(len))
print(field_catalog.groupby(['stage', 'kind']).apply(len))

Statistics for the cutouts:
stage  alpha  flavor         
1      0      dud                8362
              lensed galaxy      3388
              lensed quasar      3351
              lensing cluster    3917
       1      lensed galaxy      1898
              lensed quasar      1920
              lensing cluster    1341
2      0      dud                1129
              lensed galaxy       551
              lensing cluster      45
              test               9030
       1      lensed galaxy       146
              lensing cluster       5
dtype: int64
stage  alpha  kind
1      0      dud      8362
              sim     10656
       1      sim      5159
2      0      dud      1129
              sim       596
              test     9030
       1      sim       151
dtype: int64
Statistics for the fields:
stage  flavor         
1      dud                2971
       lensed galaxy      1879
       lensed quasar      1930
       lensing cluster    1901
2      dud                 201
  

let's examine the properties of number of markers per cluster

In [52]:
def MAD(x):
    med = np.median(x, axis=0)
    return np.median(np.abs(x - med), axis=0)

print('Mean')
print(cluster_catalog.groupby(['stage', 'alpha', 'flavor']).aggregate(np.mean)['num_markers'])
print(cluster_catalog.groupby(['stage', 'alpha', 'kind']).aggregate(np.mean)['num_markers'])

print('Median')
print(cluster_catalog.groupby(['stage', 'alpha', 'flavor']).aggregate(np.median)['num_markers'])
print(cluster_catalog.groupby(['stage', 'alpha', 'kind']).aggregate(np.median)['num_markers'])

print('MAD')
print(cluster_catalog.groupby(['stage', 'alpha', 'flavor']).aggregate(MAD)['num_markers'])
print(cluster_catalog.groupby(['stage', 'alpha', 'kind']).aggregate(MAD)['num_markers'])

Mean
stage  alpha  flavor         
1      0      dud                  3.395719
              lensed galaxy        3.561098
              lensed quasar        3.888093
              lensing cluster     19.882563
       1      lensed galaxy       78.094310
              lensed quasar      113.298438
              lensing cluster    115.826249
2      0      dud                  5.346324
              lensed galaxy        1.740472
              lensing cluster     41.111111
              test                 4.293355
       1      lensed galaxy       95.123288
              lensing cluster    307.000000
Name: num_markers, dtype: float64
stage  alpha  kind
1      0      dud       3.395719
              sim       9.663476
       1      sim     101.003877
2      0      dud       5.346324
              sim       4.713087
              test      4.293355
       1      sim     102.139073
Name: num_markers, dtype: float64
Median
stage  alpha  flavor         
1      0      dud                 1
  

now let's examine the number of clusters and the number of markers on a field

In [57]:
import pandas as pd
cluster_catalog = pd.read_csv('/Users/cpd/Projects/strongcnn/catalog/cluster_catalog.csv')
field_catalog = pd.read_csv('/Users/cpd/Projects/strongcnn/catalog/unannotated_catalog.csv')

print('Median number of Fields')
print(cluster_catalog.groupby(['ZooID']).apply(len))
zooids = field_catalog['ZooID']
field_catalog = field_catalog.set_index('ZooID')
field_catalog['num_clusters'] = cluster_catalog.groupby(['ZooID']).apply(len)
field_catalog['num_markers'] = cluster_catalog.groupby(['ZooID']).aggregate(np.sum)['num_markers']
field_catalog = field_catalog.set_index('Unnamed: 0')

print('Median number clusters')
print(field_catalog.groupby(['stage', 'flavor']).aggregate(np.median)['num_clusters'])
print(field_catalog.groupby(['stage', 'kind']).aggregate(np.median)['num_clusters'])

print('Mean number clusters')
print(field_catalog.groupby(['stage', 'flavor']).aggregate(np.mean)['num_clusters'])
print(field_catalog.groupby(['stage', 'kind']).aggregate(np.mean)['num_clusters'])

print('Median number markers')
print(field_catalog.groupby(['stage', 'flavor']).aggregate(np.median)['num_markers'])
print(field_catalog.groupby(['stage', 'kind']).aggregate(np.median)['num_markers'])

print('Mean number markers')
print(field_catalog.groupby(['stage', 'flavor']).aggregate(np.mean)['num_markers'])
print(field_catalog.groupby(['stage', 'kind']).aggregate(np.mean)['num_markers'])

Median number of Fields
ZooID
ASW0000015    8
ASW000001u    4
ASW0000028    5
ASW000002n    2
ASW000003x    3
ASW000005h    3
ASW000005m    9
ASW000006r    2
ASW000009e    3
ASW00000ah    5
ASW00000ak    9
ASW00000bw    9
ASW00000bx    2
ASW00000ch    2
ASW00000dw    8
...
ASW0009djr     4
ASW0009djs     6
ASW0009djt     4
ASW0009dju     5
ASW0009djv     5
ASW0009djw    10
ASW0009djx     4
ASW0009djy     3
ASW0009djz     6
ASW0009dk0     3
ASW0009dk1     4
ASW0009dk2     6
ASW0009dk3     6
ASW0009dk4     5
ASW0009dk5     7
Length: 12290, dtype: int64
Median number clusters
stage  flavor         
1      dud                2.0
       lensed galaxy      2.0
       lensed quasar      2.0
       lensing cluster    2.0
2      dud                5.0
       lensed galaxy      7.0
       lensing cluster    8.5
       test               2.0
Name: num_clusters, dtype: float64
stage  kind
1      dud     2
       sim     2
2      dud     5
       sim     7
       test    2
Name: num_clusters, dtype

In [21]:
# repeat for the regular catalog (by field) instead of clusters
cc_all = catalog
cc = cc_all

# now let's print some essential stats:


print(cc.columns)

# get number of objects in stage 1 vs stage 2
print(cc.groupby(['stage']).apply(len))

print(cc.groupby(['kind']).apply(len))

print(cc.groupby(['stage', 'kind']).apply(len))

# get number of flavors
print(cc.groupby(['flavor']).apply(len))


# combine together
print(cc.groupby(['alpha']).apply(len))
print(cc.groupby(['alpha', 'stage']).apply(len))
print(cc.groupby(['alpha', 'stage', 'flavor']).apply(len))

print(cc.groupby(['stage', 'flavor']).apply(len))
print(cc.groupby(['stage', 'flavor', 'status']).apply(len))

# also show median probability
print(cc.groupby(['stage', 'flavor'])['mean_probability'].apply(np.mean))
print(cc.groupby(['stage', 'flavor', 'status'])['mean_probability'].apply(np.mean))



Index([u'Unnamed: 0', u'ID', u'ZooID', u'location', u'mean_probability', u'category', u'kind', u'flavor', u'state', u'status', u'truth', u'stage', u'At_X', u'At_Y', u'PD', u'PL'], dtype='object')
stage
1        8681
2        3766
dtype: int64
kind
dud     3172
sim     5862
test    3413
dtype: int64
stage  kind
1      dud     2971
       sim     5710
2      dud      201
       sim      152
       test    3413
dtype: int64
flavor
dud                3172
lensed galaxy      2023
lensed quasar      1930
lensing cluster    1909
test               3413
dtype: int64
stage  flavor         
1      dud                2971
       lensed galaxy      1879
       lensed quasar      1930
       lensing cluster    1901
2      dud                 201
       lensed galaxy       144
       lensing cluster       8
       test               3413
dtype: int64
stage  flavor           status   
1      dud              detected        4
                        rejected     2887
                        undecided

## So in sum:

We have something like 6700 simulated lenses, 3700 known duds, 104408 'tests' (of which we can probably safely use 93000 as duds from the stage 1 rejected tests).

Data augmentation for these are clearly needed!

## Compare with knownlenses

In [120]:
# what entries in our catalog are in the knownlens ones?
kl = pd.read_csv(knownlens_path)
cc = pd.read_csv(cluster_catalog_path)

cc = cc.set_index(['ZooID'])
cc_kl = []
klifail = []
for kli in range(len(kl)):
    kx = kl['x'][kli]
    ky = 440 - kl['y'][kli]
    if kl['ZooID'][kli] in cc.index:
        cci = cc.loc[kl['ZooID'][kli]]
        ccx = cci['x']
        ccy = cci['y']
        
        l2 = np.sqrt(np.square(kx - ccx) + np.square(ky - ccy))
        if type(l2) != np.float64:
            cmin = cci.iloc[np.argmin(l2.values)]
        else:
            cc_kl.append(cci)
    else:
        klifail.append(kli)

## Augment the data: Take the images, apply a reasonable affine transformation

One guy actually did this in the convnet for the galaxy zoo morphologies -- we don't have to necessarily store 32x of these data if we don't want (also helps combat overfitting).